In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import random
from sklearn.metrics import pairwise_distances
import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA  # to apply PCA
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
import math

In [2]:
"""
This cell describes the dataset generation of HCMNIST data and this code is sourced from https://github.com/anndvision/quince.
"""

import torch
import typing

from pathlib import Path

from torchvision import datasets

from sklearn import model_selection


def lambda_top_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[k:] - mu).sum(dim=0)
    return mu + r.div(m * (alpha + 1) - k)


def lambda_bottom_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[:k] - mu).sum(dim=0)
    return mu + r.div(m * alpha + k)


def alpha_fn(pi, lambda_):
    return (pi * lambda_) ** -1 + 1.0 - lambda_ ** -1


def beta_fn(pi, lambda_):
    return lambda_ * (pi) ** -1 + 1.0 - lambda_


def policy_risk(pi, y1, y0):
    return (pi * y1 + (1 - pi) * y0).mean()

def complete_propensity(x, u, gamma, beta=0.75):
    logit = beta * x + 0.5
    nominal = (1 + np.exp(-logit)) ** -1
    alpha = alpha_fn(nominal, gamma)
    beta = beta_fn(nominal, gamma)
    return (u / alpha) + ((1 - u) / beta)


def f_mu(x, t, u, theta=4.0):
    mu = (
        (2 * t - 1) * x
        + (2.0 * t - 1)
        - 2 * np.sin((4 * t - 2) * x)
        - (theta * u - 2) * (1 + 0.5 * x)
    )
    return mu


def linear_normalization(x, new_min, new_max):
    return (x - x.min()) * (new_max - new_min) / (x.max() - x.min()) + new_min


class HCMNIST(datasets.MNIST):
    def __init__(
        self,
        root: str,
        gamma_star: float,
        split: str = "train",
        mode: str = "mu",
        p_u: str = "bernoulli",
        theta: float = 4.0,
        beta: float = 0.75,
        sigma_y: float = 1.0,
        domain: float = 2.0,
        seed: int = 1331,
        transform: typing.Optional[typing.Callable] = None,
        target_transform: typing.Optional[typing.Callable] = None,
        download: bool = True,
    ) -> None:
        train = split == "train" or split == "valid"
        root = Path.home() / "quince_datasets" if root is None else Path(root)
        self.__class__.__name__ = "MNIST"
        super(HCMNIST, self).__init__(
            root,
            train=train,
            transform=transform,
            target_transform=target_transform,
            download=download,
        )
        self.data = self.data.view(len(self.targets), -1).numpy()
        self.targets = self.targets.numpy()

        if train:
            (
                data_train,
                data_valid,
                targets_train,
                targets_valid,
            ) = model_selection.train_test_split(
                self.data, self.targets, test_size=0.3, random_state=seed
            )
            self.data = data_train if split == "train" else data_valid
            self.targets = targets_train if split == "train" else targets_valid

        self.mode = mode
        self.dim_input = [1, 28, 28]
        self.dim_treatment = 1
        self.dim_output = 1

        self.phi_model = fit_phi_model(
            root=root, edges=torch.arange(-domain, domain + 0.1, (2 * domain) / 10),
        )

        size = (self.__len__(), 1)
        rng = np.random.RandomState(seed=seed)
        if p_u == "bernoulli":
            self.u = rng.binomial(1, 0.5, size=size).astype("float32")
        elif p_u == "uniform":
            self.u = rng.uniform(size=size).astype("float32")
        elif p_u == "beta_bi":
            self.u = rng.beta(0.5, 0.5, size=size).astype("float32")
        elif p_u == "beta_uni":
            self.u = rng.beta(2, 5, size=size).astype("float32")
        else:
            raise NotImplementedError(f"{p_u} is not a supported distribution")

        phi = self.phi
        self.pi = (
            complete_propensity(x=phi, u=self.u, gamma=gamma_star, beta=beta)
            .astype("float32")
            .ravel()
        )
        self.t = rng.binomial(1, self.pi).astype("float32")
        eps = (sigma_y * rng.normal(size=self.t.shape)).astype("float32")
        self.mu0 = (
            f_mu(x=phi, t=0.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.mu1 = (
            f_mu(x=phi, t=1.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.y0 = self.mu0 + eps
        self.y1 = self.mu1 + eps
        self.y = self.t * self.y1 + (1 - self.t) * self.y0
        self.tau = self.mu1 - self.mu0
        self.y_mean = np.array([0.0], dtype="float32")
        self.y_std = np.array([1.0], dtype="float32")

    def __getitem__(self, index):
        x = ((self.data[index].astype("float32") / 255.0) - 0.1307) / 0.3081
        t = self.t[index : index + 1]
        if self.mode == "pi":
            return x, t
        elif self.mode == "mu":
            return np.hstack([x, t]), self.y[index : index + 1]
        else:
            raise NotImplementedError(
                f"{self.mode} not supported. Choose from 'pi'  for propensity models or 'mu' for expected outcome models"
            )

    @property
    def phi(self):
        x = ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081
        z = np.zeros_like(self.targets.astype("float32"))
        for k, v in self.phi_model.items():
            ind = self.targets == k
            x_ind = x[ind].reshape(ind.sum(), -1)
            means = x_ind.mean(axis=-1)
            z[ind] = linear_normalization(
                np.clip((means - v["mu"]) / v["sigma"], -1.4, 1.4), v["lo"], v["hi"]
            )
        return np.expand_dims(z, -1)

    @property
    def x(self):
        return ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081


def fit_phi_model(root, edges):
    ds = datasets.MNIST(root=root)
    data = (ds.data.float().div(255) - 0.1307).div(0.3081).view(len(ds), -1)
    model = {}
    digits = torch.unique(ds.targets)
    for i, digit in enumerate(digits):
        lo, hi = edges[i : i + 2]
        ind = ds.targets == digit
        data_ind = data[ind].view(ind.sum(), -1)
        means = data_ind.mean(dim=-1)
        mu = means.mean()
        sigma = means.std()
        model.update(
            {
                digit.item(): {
                    "mu": mu.item(),
                    "sigma": sigma.item(),
                    "lo": lo.item(),
                    "hi": hi.item(),
                }
            }
        )
    return model

In [3]:
def get_HMINIST_data(sim_num):
  obj = HCMNIST(root = "./",gamma_star = 1.1, seed=sim_num)
  # Assuming X is a matrix of size (42000, 784)
  # Assuming indices is the array of generated indices
  indices = np.random.choice(42000, size=3000, replace=False)
  # Select subrows from X using the generated indices

  X = obj.x[indices]
  Y_factual = obj.y[indices]
  T = obj.t[indices]
  mu_0 = obj.mu0[indices]
  mu_1 = obj.mu1[indices]

  X_train, X_test, Y_factual_train, _, T_train, _, mu_0_train, mu_0_test, mu_1_train, mu_1_test = train_test_split(X, Y_factual, T, mu_0, mu_1, test_size=0.1, random_state=sim_num)
  # X_train, X_test, Y_factual_train, _, T_train, _, mu_0_train, mu_0_test, mu_1_train, mu_1_test = train_test_split(X, Y_factual, T, mu_0, mu_1, test_size=0.0000001, random_state=sim_num)

  X_train = X_train.astype('float32')
  Y_factual_train = Y_factual_train.astype('float32')
  T_train = T_train.astype('float32')
  mu_0_train = mu_0_train.astype('float32')
  mu_1_train = mu_1_train.astype('float32')

  X_test = X_test.astype('float32')
  mu_0_test = mu_0_test.astype('float32')
  mu_1_test = mu_1_test.astype('float32')

  return (X_train, Y_factual_train, T_train, mu_0_train, mu_1_train), (X_test, mu_0_test, mu_1_test)

In [4]:
def random_matching_IN(X, Y, T):
  preds_IN_random = {}

  X_treated, X_control = get_separate_treated_and_control(X,T)
  Y_treated, Y_control = get_separate_treated_and_control(Y,T)


  untreated_group_selected = random.choices(range(0, len(Y_control)), k=len(Y_treated))

  treated_group_selected = random.choices(range(0, len(Y_treated)), k=len(Y_control))

  preds_IN_random['trtd_grp_y0'] = Y_control[untreated_group_selected]

  preds_IN_random['trtd_grp_y1'] = Y_treated

  preds_IN_random['cntrl_grp_y0'] = Y_control

  preds_IN_random['cntrl_grp_y1'] = Y_treated[treated_group_selected]

  return preds_IN_random

def random_matching_OUT(X_in, Y_in, T_in, X_out):
  preds_OUT_random = {}

  X_in_trtd, X_in_cntrl = get_separate_treated_and_control(X_in, T_in)
  Y_in_trtd, Y_in_cntrl = get_separate_treated_and_control(Y_in, T_in)


  untreated_group_selected = random.choices(range(0, len(Y_in_cntrl)), k=np.shape(X_out)[0] )

  treated_group_selected = random.choices(range(0, len(Y_in_trtd)), k=np.shape(X_out)[0])

  preds_OUT_random['y0'] = Y_in_cntrl[untreated_group_selected]

  preds_OUT_random['y1'] = Y_in_trtd[treated_group_selected]

  return preds_OUT_random

In [5]:
def matching_algo_with_out(distance_type, X_IN, Y_IN, T_IN, X_OUT):

  global hash_length

  preds_IN = {}
  preds_OUT = {}

  if distance_type == "random":
    preds_IN = random_matching_IN(X_IN, Y_IN, T_IN)
    preds_OUT = random_matching_OUT(X_IN, Y_IN, T_IN, X_OUT)

  else:

    X_IN_trtd, X_IN_cntrl = get_separate_treated_and_control(X_IN, T_IN)
    Y_IN_trtd, Y_IN_cntrl = get_separate_treated_and_control(Y_IN, T_IN)

    if distance_type == "hamming":
      treat_untreated_dist_IN =  np.square(pairwise_distances(X_IN_trtd, X_IN_cntrl, metric='euclidean', n_jobs=-1))/hash_length

    else:
      treat_untreated_dist_IN = pairwise_distances(X_IN_trtd, X_IN_cntrl, metric=distance_type, n_jobs=-1)

    cntrl_grp_selected_IN = np.argmin(treat_untreated_dist_IN, axis=1)
    trtd_grp_selected_IN = np.argmin(treat_untreated_dist_IN, axis=0)

    preds_IN['trtd_grp_y0'] = Y_IN_cntrl[cntrl_grp_selected_IN]

    preds_IN['trtd_grp_y1'] = Y_IN_trtd

    preds_IN['cntrl_grp_y0'] = Y_IN_cntrl

    preds_IN['cntrl_grp_y1'] = Y_IN_trtd[trtd_grp_selected_IN]

    # Doing prediction on outsample test

    if distance_type == "hamming":
      X_OUT_vs_X_IN_cntrl_dist = np.square(pairwise_distances(X_OUT, X_IN_cntrl, metric='euclidean', n_jobs=-1))/hash_length
      X_OUT_vs_X_IN_trtd_dist = np.square(pairwise_distances(X_OUT, X_IN_trtd, metric='euclidean', n_jobs=-1))/hash_length

    else:
      X_OUT_vs_X_IN_cntrl_dist = pairwise_distances(X_OUT, X_IN_cntrl, metric=distance_type, n_jobs=-1)
      X_OUT_vs_X_IN_trtd_dist = pairwise_distances(X_OUT, X_IN_trtd, metric=distance_type, n_jobs=-1)

    cntrl_grp_selected_OUT = np.argmin(X_OUT_vs_X_IN_cntrl_dist, axis=1)
    trtd_grp_selected_OUT = np.argmin(X_OUT_vs_X_IN_trtd_dist, axis=1)

    preds_OUT['y0'] = Y_IN_cntrl[cntrl_grp_selected_OUT]

    preds_OUT['y1'] = Y_IN_trtd[trtd_grp_selected_OUT]

  return preds_IN, preds_OUT

In [6]:
class Eval_metrics_In_sample():
    #EVAL METRICS FOR INSMAPLES ----------------
    def __init__(self,mu_0, mu_1, T):
        self.data={}

        mu_0_trted, mu_0_cntrl = get_separate_treated_and_control(mu_0, T)

        mu_1_trted, mu_1_cntrl = get_separate_treated_and_control(mu_1, T)

        self.data['trtd_grp_mu_0'] = mu_0_trted
        self.data['cntrl_grp_mu_0'] = mu_0_cntrl

        self.data['trtd_grp_mu_1'] = mu_1_trted
        self.data['cntrl_grp_mu_1'] = mu_1_cntrl

    def ATE_absolute_error(self,preds):

        ITTs_pred =  preds['trtd_grp_y1'] - preds['trtd_grp_y0']
        ITUs_pred =  preds['cntrl_grp_y1'] - preds['cntrl_grp_y0']
        ATE_pred = np.mean(np.concatenate((ITTs_pred, ITUs_pred)))

        ITTs_true =  self.data['trtd_grp_mu_1']  - self.data['trtd_grp_mu_0']
        ITUs_true =  self.data['cntrl_grp_mu_1']  - self.data['cntrl_grp_mu_0']
        ATE_true = np.mean(np.concatenate((ITTs_true, ITUs_true)))

        # print("TrueATE:- ",ATE_true)
        # print("PredATE:- ",ATE_pred)
        # print("--------------------")
        return np.abs(ATE_true- ATE_pred)


    def ITE_RMSE_func(self,preds):

        ITTs_pred =  preds['trtd_grp_y1'] - preds['trtd_grp_y0']
        ITUs_pred =  preds['cntrl_grp_y1'] - preds['cntrl_grp_y0']
        ITE_pred = np.concatenate((ITTs_pred, ITUs_pred))

        ITTs_true =  self.data['trtd_grp_mu_1']  - self.data['trtd_grp_mu_0']
        ITUs_true =  self.data['cntrl_grp_mu_1']  - self.data['cntrl_grp_mu_0']
        ITE_true = np.concatenate((ITTs_true, ITUs_true))

        ITE_RMSE = np.sqrt(np.mean(np.square(ITE_true - ITE_pred)))

        return ITE_RMSE

In [7]:

class Eval_metrics_Out_sample():
    #EVAL METRICS FOR OUT SMAPLES -----------------
    def __init__(self,mu_0, mu_1):
        self.data={}

        self.data['mu_0'] = mu_0
        self.data['mu_1'] = mu_1

    def ATE_absolute_error(self,preds):

        ATE_pred = np.mean(preds['y1'] - preds['y0'])
        ATE_true = np.mean(self.data['mu_1'] - self.data['mu_0'])



        return np.abs(ATE_true- ATE_pred)

    def PEHE_func(self,preds):

        PEHE = np.sqrt(np.mean(( np.square( ( (self.data['mu_1']-self.data['mu_0']) - (preds['y1']-preds['y0']) ) ) )))

        return PEHE

In [8]:
def get_separate_treated_and_control(Q,T):
  Q_treated = Q[T.astype(bool)]
  Q_control = Q[np.logical_not(T.astype(bool))]
  return Q_treated, Q_control

def perform_PCA(IN_data_matrix, OUT_data_matrix):
  pca = PCA(n_components=5)
  pca.fit(IN_data_matrix)
  Z_IN = pca.transform(IN_data_matrix)
  Z_OUT = pca.transform(OUT_data_matrix)
  return Z_IN, Z_OUT

def create_hash(IN_data_matrix, OUT_data_matrix):
  global hash_length
  mid = (int) (hash_length/2)
  random_matrix = np.random.randn(hash_length,np.shape(IN_data_matrix)[1]).T
  lmbda = np.random.uniform(low=-5, high=5, size=(1,mid))

  Z_IN = np.dot(IN_data_matrix, random_matrix)



  Z_IN[:,mid:] = np.add(Z_IN[:,mid:], np.tile(lmbda, (np.shape(Z_IN)[0], 1)))
  Z_IN = Z_IN > 0
  Z_IN = Z_IN.astype(int)

  Z_OUT = np.dot(OUT_data_matrix, random_matrix)
  Z_OUT[:,mid:] = np.add(Z_OUT[:,mid:], np.tile(lmbda, (np.shape(Z_OUT)[0], 1)))
  Z_OUT = Z_OUT > 0
  Z_OUT = Z_OUT.astype(int)
  return Z_IN, Z_OUT

In [9]:
hash_length = 10000 # Hash vector length
def select_baseline(choice, covs_in, treats_in, covs_out):
  if choice == 1:
    return  "random", covs_in, covs_out

  elif choice == 2:
    return "l2", covs_in, covs_out

  elif choice == 3:
    covs_in, covs_out = perform_PCA(covs_in, covs_out)
    return "l2", covs_in, covs_out



  elif choice == 4:
    propensity_score_model_X = LogisticRegression(solver='liblinear', class_weight = "balanced").fit(covs_in, treats_in)
    e_X_in = propensity_score_model_X.predict_proba(covs_in)[:,0]
    e_X_out = propensity_score_model_X.predict_proba(covs_out)[:,0]

    e_X_in = e_X_in.reshape(-1,1)
    e_X_out = e_X_out.reshape(-1,1)
    return "l1", e_X_in, e_X_out

  elif choice == 5:
    covs_in, covs_out = perform_PCA(covs_in, covs_out)
    propensity_score_model_Z = LogisticRegression(solver='liblinear', class_weight = "balanced").fit(covs_in, treats_in)

    e_Z_in = propensity_score_model_Z.predict_proba(covs_in)[:,0]
    e_Z_out = propensity_score_model_Z.predict_proba(covs_out)[:,0]

    e_Z_in = e_Z_in.reshape(-1,1)
    e_Z_out = e_Z_out.reshape(-1,1)
    return "l1", e_Z_in, e_Z_out

  elif choice == 6:
    covs_in, covs_out = create_hash(covs_in, covs_out)
    return "hamming", covs_in, covs_out

  else:
    print("WRONG CHOICE")

In [21]:
def run_exp(choice):
  error_ATE_in_sample =[]
  RMSE_ITE_in_sample =[]

  error_ATE_out_sample =[]
  PEHE_out_sample =[]
  dataset_sim_num = 100
  choice_map = {1:"Random", 2:"X", 3:"PCA(X)", 4:"e(X)", 5:"e(PCA(X))", 6:"RHPT"} # e() is propensity score
  for i in tqdm(range(1, dataset_sim_num+1), desc="HCMNIST Dataset Sim-Experiments Progress"):

    np.random.seed(i)
    train_data, test_data = get_HMINIST_data(i)

    X_in, Y_in, T_in, mu_0_in, mu_1_in = train_data

    X_out, mu_0_out, mu_1_out = test_data

    distance_type, X_in, X_out = select_baseline(choice, X_in, T_in, X_out)

    preds_in, preds_out = matching_algo_with_out(distance_type, X_in, Y_in, T_in, X_out)

    in_sample_evals = Eval_metrics_In_sample(mu_0 = mu_0_in, mu_1 = mu_1_in, T = T_in)
    out_sample_evals = Eval_metrics_Out_sample(mu_0=mu_0_out, mu_1=mu_1_out)

    error_ATE_in_sample.append(in_sample_evals.ATE_absolute_error(preds_in))
    RMSE_ITE_in_sample.append(in_sample_evals.ITE_RMSE_func(preds_in))

    error_ATE_out_sample.append(out_sample_evals.ATE_absolute_error(preds_out))
    PEHE_out_sample.append(out_sample_evals.PEHE_func(preds_out))

  print("In-sample ATE absolute error for ",choice_map[choice]," matching =", round(np.mean(error_ATE_in_sample),2),"+-", round((np.std(error_ATE_in_sample, ddof=1) / np.sqrt(np.size(error_ATE_in_sample))),2))
  print("In-sample ITE RMSE for ",choice_map[choice]," matching =", round(np.mean(RMSE_ITE_in_sample),2),"+-", round((np.std(RMSE_ITE_in_sample, ddof=1) / np.sqrt(np.size(RMSE_ITE_in_sample))),2))
  print("Out-sample ATE error for ",choice_map[choice]," matching =", round(np.mean(error_ATE_out_sample),2),"+-", round((np.std(error_ATE_out_sample, ddof=1) / np.sqrt(np.size(error_ATE_out_sample))),2))
  print("Out-sample PEHE error for ",choice_map[choice]," matching =", round(np.mean(PEHE_out_sample),2),"+-", round((np.std(PEHE_out_sample, ddof=1) / np.sqrt(np.size(PEHE_out_sample))),2))


In [22]:
start_time = time.time()
# choice_map = {1:"Random", 2:"X", 3:"PCA(X)", 4:"e(X)", 5:"e(PCA(X))", 6:"RHPT"} # e() is propensity score
run_exp(choice=6) # Change choice for which baseline do you want
end_time = time.time()
elapsed_time_seconds = end_time - start_time
# Convert elapsed time to minutes
elapsed_time_minutes = elapsed_time_seconds / 60
print("Time Taken in Minutes", elapsed_time_minutes)

HCMNIST Dataset Sim-Experiments Progress:   0%|          | 0/100 [00:00<?, ?it/s]

In-sample ATE absolute error for  RHPT  matching = 0.19 +- 0.01
In-sample ITE RMSE for  RHPT  matching = 3.6 +- 0.01
Out-sample ATE error for  RHPT  matching = 0.23 +- 0.02
Out-sample PEHE error for  RHPT  matching = 3.75 +- 0.02
Time Taken in Minutes 9.359491340319316
